# SOM (Koholen Maps)

SOM (Self-Organizing Maps) is a type of artificial neural network that is trained using unsupervised learning to produce a low-dimensional (typically two-dimensional), discretized representation of the input space of the training samples. The SOM algorithm uses competitive learning to train the neurons to represent the input data in a way that preserves the topological properties of the input space. The SOM algorithm is also known as Kohonen Maps, named after Teuvo Kohonen, the inventor of the algorithm.

## Setup

### Set Random State Seed

In [1]:
import random
import numpy as np

seed = 42
random.seed(seed)
np.random.seed(seed)

### Import ML lib

In [2]:
import os
import sys

project_root = os.path.abspath(
    os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir)
)
if project_root not in sys.path:
    sys.path.append(project_root)

try:
    from src.data_loader.mnist_dataset import load_mnist_data
    from src.data_loader.toy_dataset import load_toy_data
    from src.algorithms.som.som import SOM
except ModuleNotFoundError as e:
    print(e)

2024-06-26 22:11:36.928259: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 22:11:38.077308: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Test on toy data

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

X, y = load_toy_data(n_samples=1000, n_features=5, n_clusters=3)
X = StandardScaler().fit_transform(X)

som = SOM(x=10, y=10, input_len=3, learning_rate=0.5, seed=42)
som.train(X, num_iterations=100)

mapped = som.map_vects(X)

plt.scatter(mapped[:, 0], mapped[:, 1])
plt.title("SOM Mapping")
plt.show()

ValueError: operands could not be broadcast together with shapes (10,10,3) (5,) 

## Load MNIST Dataset